# Statistical Modelling of total contacts: model fit
#### Non negative binomial model with EPID random effect

## Models for Adult respondents

In [2]:
# # Adults
library(colorspace)
library(MASS)
library(glmmTMB)
library(dplyr)
library(vctrs)
library(ggplot2)
library(shape)
library(stringr)
library(RColorBrewer)
library(huxtable)
library(texreg)
library(tidyverse)
library(ggstats)
library(ggforce)
library(ggstance)
library(broom.mixed)
# library(Polychrome)
library(Hmisc)
library(showtext)
library(haven)

font_add_google(name = "Barlow", family = "Barlow")
showtext_auto()

# # ```{r data}
load("data_in/datamod.Rdata")

# ```{r adults}
data_for_model = cbind(data_mod[,c("total_contacts", "total_contacts_nohh", "total_contacts_prol", "total_contacts_prol_soft", "total_contacts_nohh_prol","wave","EPID")],data_mod[,1:34])
names(data_for_model)[1:7]=c("total_contacts", "total_contacts_nohh", "total_contacts_prol", "total_contacts_prol_soft","total_contacts_nohh_prol","wave","EPID")

### Prepare "data_fo_model_prol"
data_for_model_prol <- data_for_model[data_for_model$total_contacts_prol <= 100,] # censoring at 100
Hmisc::label(data_for_model_prol) =  as.list(var.labels[match(names(data_for_model_prol), names(var.labels))])


Attaching package: ‘dplyr’


The following object is masked from ‘package:MASS’:

    select


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘vctrs’


The following object is masked from ‘package:dplyr’:

    data_frame


Warning message:
“package ‘stringr’ was built under R version 4.5.2”
Warning message:
“package ‘huxtable’ was built under R version 4.5.2”

Attaching package: ‘huxtable’


The following object is masked from ‘package:ggplot2’:

    theme_grey


The following object is masked from ‘package:dplyr’:

    add_rownames


Version:  1.39.4
Date:     2024-07-23
Author:   Philip Leifeld (University of Manchester)

Consider submitting praise using the praise or praise_interactive functions.
Please cite the JSS article in your publications -- see citation("texreg").

Warning message:
“package ‘purrr’ was built under R version 4.5.2”
──

In [3]:
colnames(data_for_model)

[1] "total_contacts"           "total_contacts_nohh"     
 [3] "total_contacts_prol"      "total_contacts_prol_soft"
 [5] "total_contacts_nohh_prol" "wave"                    
 [7] "EPID"                     "region_grouped_IT"       
 [9] "respondent_gender"        "education"               
[11] "occupation_agg"           "d_occupation"            
[13] "occupation_agg2"          "sunday"                  
[15] "weekend"                  "worked_in_presence"      
[17] "attend_in_presence"       "d_children"              
[19] "d_senior65"               "d_senior70"              
[21] "d_senior75"               "d_senior80"              
[23] "d_hh_size"                "hh_size"                 
[25] "hh_size_det"              "chronic_comorb_self2"    
[27] "chronic_comorb_cohab2"    "covid"                   
[29] "time_since_covid"         "d_time_since_covid"      
[31] "vacc_covid_bin"           "d_vacc2"                 
[33] "vacc_covid_bin_2"         "other_vacc"              
[35] "refusal"                  "perceived_income"        
[37] "income_1000"              "income"                  
[39] "income_threecat"          "knows_severe"            
[41] "age_group_10"

In [4]:
# A1) direct+indirect contacts with cohabitants
adults_indirect_totcont = glmmTMB(total_contacts_prol ~ wave + (1|EPID) +
                                age_group_10+
                                education+
                                occupation_agg+
                                income_threecat+
                                attend_in_presence+
                                sunday+
                                d_vacc2+
                                d_time_since_covid + 
                                chronic_comorb_self2 +
                                chronic_comorb_cohab2+
                                d_senior65+
                                d_children +
                                hh_size,
                              family=nbinom2,
                              control=glmmTMBControl(optimizer=optim,optArgs=list(method="BFGS")),
                         data=data_for_model_prol) 
summary(adults_indirect_totcont)
# Save model fit to RDS
saveRDS(adults_indirect_totcont, file = "model_out/adults_indirect_totcont_glmmTMB.rds")

 Family: nbinom2  ( log )
Formula:          
total_contacts_prol ~ wave + (1 | EPID) + age_group_10 + education +  
    occupation_agg + income_threecat + attend_in_presence + sunday +  
    d_vacc2 + d_time_since_covid + chronic_comorb_self2 + chronic_comorb_cohab2 +  
    d_senior65 + d_children + hh_size
Data: data_for_model_prol

     AIC      BIC   logLik deviance df.resid 
 25454.0  25646.1 -12697.0  25394.0     4435 

Random effects:

Conditional model:
 Groups Name        Variance Std.Dev.
 EPID   (Intercept) 0.4648   0.6817  
Number of obs: 4465, groups:  EPID, 3309

Dispersion parameter for nbinom2 family (): 3.22 

Conditional model:
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     0.985738   0.114679   8.596  < 2e-16 ***
wave2                           0.108975   0.026078   4.179 2.93e-05 ***
age_group_1030-39 y            -0.135479   0.073154  -1.852 0.064028 .  
age_group_1040-49 y            -0.352158   0.075943  -4

In [4]:
# A2) direct+indirect contacts without cohabitants
adults_indirect_ncohabcont = glmmTMB(total_contacts_nohh_prol ~ wave + (1|EPID) +
                                respondent_gender+
                                age_group_10+
                                education+
                                occupation_agg+
                                income_threecat+
                                attend_in_presence+
                                sunday+
                                d_vacc2+
                                d_time_since_covid + 
                                chronic_comorb_self2 +
                                chronic_comorb_cohab2+
                                d_senior65+
                                d_children +
                                hh_size,
                       family=nbinom2,
                       control=glmmTMBControl(optimizer=optim,optArgs=list(method="BFGS")),
                       data=data_for_model_prol)
summary(adults_indirect_ncohabcont)
# Save model fit to RDS
saveRDS(adults_indirect_ncohabcont, file = "model_out/adults_indirect_ncohabcont_glmmTMB.rds")

 Family: nbinom2  ( log )
Formula:          
total_contacts_nohh_prol ~ wave + (1 | EPID) + respondent_gender +  
    age_group_10 + education + occupation_agg + income_threecat +  
    attend_in_presence + sunday + d_vacc2 + d_time_since_covid +  
    chronic_comorb_self2 + chronic_comorb_cohab2 + d_senior65 +  
    d_children + hh_size
Data: data_for_model_prol

     AIC      BIC   logLik deviance df.resid 
   22202    22398   -11070    22140     4084 

Random effects:

Conditional model:
 Groups Name        Variance Std.Dev.
 EPID   (Intercept) 0.6834   0.8267  
Number of obs: 4115, groups:  EPID, 3088

Dispersion parameter for nbinom2 family (): 1.44 

Conditional model:
                               Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     1.23297    0.16392   7.522 5.41e-14 ***
wave2                           0.13145    0.03814   3.446 0.000569 ***
respondent_genderFemale        -0.05330    0.04945  -1.078 0.281075    
age_group_1030-39 y         

In [5]:
# B1) direct contacts (only) with cohabitants
adults_orig_totcont = glmmTMB(total_contacts ~ wave + (1|EPID) +
                                respondent_gender+
                                age_group_10+
                                education+
                                occupation_agg+
                                income_threecat+
                                attend_in_presence+
                                sunday+
                                d_vacc2+
                                d_time_since_covid + 
                                chronic_comorb_self2 +
                                chronic_comorb_cohab2+
                                d_senior65+
                                d_children +
                                hh_size,
                        control=glmmTMBControl(optimizer=optim,optArgs=list(method="BFGS")),
                        family=nbinom2,
                        data=data_for_model_prol)
summary(adults_orig_totcont)
# Save model fit to RDS
saveRDS(adults_orig_totcont, file = "model_out/adults_orig_totcont_glmmTMB.rds")

 Family: nbinom2  ( log )
Formula:          
total_contacts ~ wave + (1 | EPID) + respondent_gender + age_group_10 +  
    education + occupation_agg + income_threecat + attend_in_presence +  
    sunday + d_vacc2 + d_time_since_covid + chronic_comorb_self2 +  
    chronic_comorb_cohab2 + d_senior65 + d_children + hh_size
Data: data_for_model_prol

     AIC      BIC   logLik deviance df.resid 
 21795.9  21994.4 -10867.0  21733.9     4434 

Random effects:

Conditional model:
 Groups Name        Variance Std.Dev.
 EPID   (Intercept) 0.2618   0.5117  
Number of obs: 4465, groups:  EPID, 3309

Dispersion parameter for nbinom2 family (): 9.96 

Conditional model:
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     0.527349   0.092094   5.726 1.03e-08 ***
wave2                           0.088922   0.020454   4.347 1.38e-05 ***
respondent_genderFemale         0.051806   0.027871   1.859  0.06306 .  
age_group_1030-39 y             0.002281

In [6]:
# B2) direct+indirect soft contacts with cohabitants
adults_soft_totcont = glmmTMB(total_contacts_prol_soft ~ wave + (1|EPID) +
                                respondent_gender+
                                age_group_10+
                                education+
                                occupation_agg+
                                income_threecat+
                                attend_in_presence+
                                sunday+
                                d_vacc2+
                                d_time_since_covid + 
                                chronic_comorb_self2 +
                                chronic_comorb_cohab2+
                                d_senior65+
                                d_children +
                                hh_size,
                        control=glmmTMBControl(optimizer=optim, optArgs=list(method="BFGS")),
                        family=nbinom2,
                        data=data_for_model_prol)
summary(adults_soft_totcont)
# Save model fit to RDS
saveRDS(adults_soft_totcont, file = "model_out/adults_soft_totcont_glmmTMB.rds")

 Family: nbinom2  ( log )
Formula:          
total_contacts_prol_soft ~ wave + (1 | EPID) + respondent_gender +  
    age_group_10 + education + occupation_agg + income_threecat +  
    attend_in_presence + sunday + d_vacc2 + d_time_since_covid +  
    chronic_comorb_self2 + chronic_comorb_cohab2 + d_senior65 +  
    d_children + hh_size
Data: data_for_model_prol

     AIC      BIC   logLik deviance df.resid 
 28411.4  28610.0 -14174.7  28349.4     4434 

Random effects:

Conditional model:
 Groups Name        Variance Std.Dev.
 EPID   (Intercept) 0.3923   0.6263  
Number of obs: 4465, groups:  EPID, 3309

Dispersion parameter for nbinom2 family (): 3.15 

Conditional model:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     1.2692412  0.1085703  11.690  < 2e-16 ***
wave2                           0.1228912  0.0244218   5.032 4.85e-07 ***
respondent_genderFemale         0.0403316  0.0333467   1.209 0.226485    
age_group_1030-39 y 

## Models for Underage respondents

In [7]:
# # Children
# ```{r children}
data_mod = data_mod_children %>% 
  rename(parental_education = education,
         parental_occupation= occupation_agg2
         )
data_mod$parental_occupation[data_mod$respondent_age>15]=NA #set to NA for 16-17 they answered about themselves
data_mod$parental_occupation=ifelse(data_mod$parental_occupation %in% c("Employed","Student"),"Employed","Other (not working)")
data_mod$kindergarten_05[data_mod$respondent_age>5]="No"
data_mod$hh_size = droplevels(data_mod$hh_size)
data_mod <- arrange(data_mod,EPID,wave)
data_mod$presence_school <- ifelse(data_mod$presence_school == "In presence", "In presence", "Dind't attend")
data_mod$presence_school <- factor(data_mod$presence_school)

data_mod$age_group_child <- ifelse(data_mod$respondent_age <= 5, "0-5 y", 
                                   ifelse(data_mod$respondent_age > 5 & data_mod$respondent_age <=13, "6-13 y",
                                          ifelse(data_mod$respondent_age >13, "14-17 y", NA)))
data_mod$age_group_child <- factor(data_mod$age_group_child, levels = c("0-5 y", "6-13 y", "14-17 y"))
data_mod$age_group_child <- relevel(data_mod$age_group_child, ref = 2)

var.labels = c(var.labels,
               age_group_child = "Age group of the child",
               parental_occupation = "Parental occupation",
               parental_education = "Parental education",
               presence_school = "Went to school is presence",
               hh_size = "Household size")
Hmisc::label(data_mod) =  as.list(var.labels[match(names(data_mod), names(var.labels))])

data_for_model_prol <- data_mod[data_mod$total_contacts_prol <= 100 & data_mod$total_contacts_prol!=0,] # censoring at 100

In [8]:
colnames(data_mod)

[1] "region_grouped_IT"            "respondent_gender"           
 [3] "parental_education"           "occupation_agg"              
 [5] "d_occupation"                 "parental_occupation"         
 [7] "sunday"                       "weekend"                     
 [9] "worked_in_presence"           "attend_in_presence"          
[11] "d_children"                   "d_senior65"                  
[13] "d_senior70"                   "d_senior75"                  
[15] "d_senior80"                   "d_hh_size"                   
[17] "hh_size"                      "hh_size_det"                 
[19] "chronic_comorb_self2"         "chronic_comorb_cohab2"       
[21] "covid"                        "time_since_covid"            
[23] "d_time_since_covid"           "vacc_covid_bin"              
[25] "d_vacc2"                      "vacc_covid_bin_2"            
[27] "other_vacc"                   "refusal"                     
[29] "perceived_income"             "income_1000"                 
[31] "income"                       "income_threecat"             
[33] "knows_severe"                 "age_group_10"                
[35] "age_group_25"                 "wave"                        
[37] "EPID"                         "age"                         
[39] "kindergarten_05"              "presence_school"             
[41] "total_contacts"               "c_sharedindoor"              
[43] "respondent_age"               "indoor_nocohab"              
[45] "indoor_mask_nocohab"          "indoor_nomask_nocohab"       
[47] "indoor_nocohab_notransp"      "indoor_mask_nocohab_notransp"
[49] "outdoor_nocohab"              "outdoor_mask_nocohab"        
[51] "outdoor_nomask_nocohab"       "risky"                       
[53] "total_contacts_prol"          "total_contacts_prol_soft"    
[55] "total_contacts_nohh_prol"     "total_contacts_nohh"         
[57] "age_group_child"

In [9]:
# A) children direct+indirect contacts
children_indirect_totcont = glmmTMB(total_contacts_prol ~ wave + (1|EPID) +
                                  respondent_gender+
                                  age_group_child +
                                  parental_education +
                                  parental_occupation+
                                  income_threecat+
                                  presence_school +
                                  kindergarten_05 +
                                  sunday+
                                  d_time_since_covid +
                                  d_vacc2 +
                                  other_vacc +
                                  chronic_comorb_self2+
                                  chronic_comorb_cohab2+
                                  d_senior65+
                                  hh_size
                         + d_vacc2:wave,
                       family=nbinom2,
                       data=data_for_model_prol)
summary(children_indirect_totcont)
# Save model fit to RDS
saveRDS(children_indirect_totcont, file = "model_out/children_indirect_totcont_glmmTMB.rds")

 Family: nbinom2  ( log )
Formula:          
total_contacts_prol ~ wave + (1 | EPID) + respondent_gender +  
    age_group_child + parental_education + parental_occupation +  
    income_threecat + presence_school + kindergarten_05 + sunday +  
    d_time_since_covid + d_vacc2 + other_vacc + chronic_comorb_self2 +  
    chronic_comorb_cohab2 + d_senior65 + hh_size + d_vacc2:wave
Data: data_for_model_prol

     AIC      BIC   logLik deviance df.resid 
  3480.8   3595.3  -1713.4   3426.8      487 

Random effects:

Conditional model:
 Groups Name        Variance Std.Dev.
 EPID   (Intercept) 0.1219   0.3491  
Number of obs: 514, groups:  EPID, 439

Dispersion parameter for nbinom2 family ():    3 

Conditional model:
                                        Estimate Std. Error z value Pr(>|z|)
(Intercept)                             1.456053   0.273339   5.327 9.99e-08
wave2                                   0.249810   0.102470   2.438 0.014773
respondent_genderFemale                 0.069

In [10]:
# A2) children direct+indirect contacts without cohabitants
children_indirect_ncohabcont = glmmTMB(total_contacts_nohh_prol ~ wave + (1|EPID) +
                          respondent_gender+
                                  age_group_child +
                                  parental_education +
                                  parental_occupation+
                                  income_threecat+
                                  presence_school +
                                  kindergarten_05 +
                                  sunday+
                                  d_time_since_covid +
                                  d_vacc2 +
                                  other_vacc +
                                  chronic_comorb_self2+
                                  chronic_comorb_cohab2+
                                  d_senior65+
                                  hh_size
                         + d_vacc2:wave,
                       family=nbinom2,
                       data=data_for_model_prol)
summary(children_indirect_ncohabcont)
# Save model fit to RDS
saveRDS(children_indirect_ncohabcont, file = "model_out/children_indirect_ncohabcont_glmmTMB.rds")

 Family: nbinom2  ( log )
Formula:          
total_contacts_nohh_prol ~ wave + (1 | EPID) + respondent_gender +  
    age_group_child + parental_education + parental_occupation +  
    income_threecat + presence_school + kindergarten_05 + sunday +  
    d_time_since_covid + d_vacc2 + other_vacc + chronic_comorb_self2 +  
    chronic_comorb_cohab2 + d_senior65 + hh_size + d_vacc2:wave
Data: data_for_model_prol

     AIC      BIC   logLik deviance df.resid 
  3159.7   3273.2  -1552.9   3105.7      466 

Random effects:

Conditional model:
 Groups Name        Variance  Std.Dev. 
 EPID   (Intercept) 1.078e-08 0.0001038
Number of obs: 493, groups:  EPID, 422

Dispersion parameter for nbinom2 family (): 0.907 

Conditional model:
                                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)                             0.95274    0.39365   2.420 0.015509 *  
wave2                                   0.36991    0.15956   2.318 0.020428 *  
respondent_genderFemale   

In [11]:
# B1) children direct contacts (only) with cohabitants
children_orig_totcont = glmmTMB(total_contacts ~ wave + (1|EPID) +
                          respondent_gender+
                                  age_group_child +
                                  parental_education +
                                  parental_occupation+
                                  income_threecat+
                                  presence_school +
                                  kindergarten_05 +
                                  sunday+
                                  d_time_since_covid +
                                  d_vacc2 +
                                  other_vacc +
                                  chronic_comorb_self2+
                                  chronic_comorb_cohab2+
                                  d_senior65+
                                  hh_size
                         + d_vacc2:wave,
                       family=nbinom2,
                       data=data_for_model_prol)
summary(children_orig_totcont)
# Save model fit to RDS
saveRDS(children_orig_totcont, file = "model_out/children_orig_totcont_glmmTMB.rds")

 Family: nbinom2  ( log )
Formula:          
total_contacts ~ wave + (1 | EPID) + respondent_gender + age_group_child +  
    parental_education + parental_occupation + income_threecat +  
    presence_school + kindergarten_05 + sunday + d_time_since_covid +  
    d_vacc2 + other_vacc + chronic_comorb_self2 + chronic_comorb_cohab2 +  
    d_senior65 + hh_size + d_vacc2:wave
Data: data_for_model_prol

     AIC      BIC   logLik deviance df.resid 
  2901.3   3015.8  -1423.6   2847.3      487 

Random effects:

Conditional model:
 Groups Name        Variance Std.Dev.
 EPID   (Intercept) 0.1344   0.3666  
Number of obs: 514, groups:  EPID, 439

Dispersion parameter for nbinom2 family (): 5.24 

Conditional model:
                                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)                             0.55707    0.25940   2.148 0.031752 *  
wave2                                   0.09713    0.09404   1.033 0.301673    
respondent_genderFemale                 0

In [12]:
# B2) children direct+indirect soft contacts (only) with cohabitants
children_soft_totcont = glmmTMB(total_contacts_prol_soft ~ wave + (1|EPID) +
                          respondent_gender+
                                  age_group_child +
                                  parental_education +
                                  parental_occupation+
                                  income_threecat+
                                  presence_school +
                                  kindergarten_05 +
                                  sunday+
                                  d_time_since_covid +
                                  d_vacc2 +
                                  other_vacc +
                                  chronic_comorb_self2+
                                  chronic_comorb_cohab2+
                                  d_senior65+
                                  hh_size
                         + d_vacc2:wave,
                       family=nbinom2,
                       data=data_for_model_prol)
summary(children_soft_totcont)
# Save model fit to RDS
saveRDS(children_soft_totcont, file = "model_out/children_soft_totcont_glmmTMB.rds")

 Family: nbinom2  ( log )
Formula:          
total_contacts_prol_soft ~ wave + (1 | EPID) + respondent_gender +  
    age_group_child + parental_education + parental_occupation +  
    income_threecat + presence_school + kindergarten_05 + sunday +  
    d_time_since_covid + d_vacc2 + other_vacc + chronic_comorb_self2 +  
    chronic_comorb_cohab2 + d_senior65 + hh_size + d_vacc2:wave
Data: data_for_model_prol

     AIC      BIC   logLik deviance df.resid 
  3804.5   3919.0  -1875.2   3750.5      487 

Random effects:

Conditional model:
 Groups Name        Variance Std.Dev.
 EPID   (Intercept) 0.101    0.3178  
Number of obs: 514, groups:  EPID, 439

Dispersion parameter for nbinom2 family ():  3.2 

Conditional model:
                                        Estimate Std. Error z value Pr(>|z|)
(Intercept)                             1.707307   0.253026   6.748 1.50e-11
wave2                                   0.196446   0.095114   2.065 0.038888
respondent_genderFemale                 